In [1]:
# Importer les bibliothèques nécessaires
import osmnx as ox  # Bibliothèque pour récupérer et visualiser les réseaux de rues à partir d'OpenStreetMap
import networkx as nx  # Bibliothèque pour créer et gérer des réseaux/graphes complexes
from shapely.geometry import Point, LineString, MultiLineString  # Pour les opérations géométriques
from scipy.spatial import distance  # Pour les calculs de distance spatiale
from geopy.distance import geodesic  # Pour les calculs de distance géodésique (à la surface de la Terre)
from neo4j import GraphDatabase  # Pilote de base de données Neo4j
import numpy as np  # Pour les opérations numériques


# Préparation de graphes pour stockage dans Neo4j

In [1]:
def calculate_distance(point1, point2):
    """
    Calcule la distance euclidienne entre deux points.
    :param point1: Tuple (x, y) du premier point.
    :param point2: Tuple (x, y) du deuxième point.
    :return: Distance euclidienne entre les deux points.
    """
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def calculate_distance_in_meters(point1, point2):
    # Assuming this is a placeholder function, implement it as needed
    return calculate_distance(point1, point2) * 1000
def add_pharmacy_node(G,osmid,pharmacy_point):
    """
    Ajoute un nœud de pharmacie au graphe OSMnx.
    :param G: Le graphe OSMnx du réseau routier.
    :param pharmacy_point: Un objet Point shapely avec les coordonnées de la pharmacie.
    :param attributes: Attributs supplémentaires à ajouter au nœud.
    :return: L'identifiant du nouveau nœud créé.
    """
    # Ajouter le nœud avec les attributs spécifiés
    print('osmid : ' , osmid)
    G.add_node(osmid , x=pharmacy_point.x, y=pharmacy_point.y, osmid=osmid)

    # Ajouter des attributs spécifiques à la pharmacie
    G.nodes[osmid]['pharmacy'] = 'yes'
def calculate_linestring_distance_in_meters(coords):
    total_distance = 0.0
    
    # Itérer sur les paires de points consécutives
    for i in range(len(coords) - 1):
        point1 = coords[i]
        point2 = coords[i + 1]
        
        # Calculer la distance entre les deux points en mètres
        distance = geodesic(point1, point2).meters
        total_distance += distance
    
    return total_distance
def calculate_distance(coord1,coord2):
    # Calculate the distance
    distance = geodesic(coord1, coord2).meters
    return distance
def linestring_to_list(geometry):
    if isinstance(geometry, LineString):
        return list(geometry.coords)
    return None
def projection_point_on_line(x1, y1, x2, y2, x3, y3):
    # Vecteur directeur de AB
    ABx = x2 - x1
    ABy = y2 - y1
    
    # Vecteur AC
    ACx = x3 - x1
    ACy = y3 - y1
    
    # Produit scalaire de AC et AB
    dot_product = ACx * ABx + ACy * ABy
    
    # Norme au carré de AB
    norm_squared = ABx * ABx + ABy * ABy
    
    # Calcul du paramètre t
    t = dot_product / norm_squared
    
    # Coordonnées de la projection de C sur la droite AB
    Px = x1 + t * ABx
    Py = y1 + t * ABy
    
    return Px, Py
# 1. Trouver le point  le plus proche pour pharmacie

def find_nearest_point(pharmacy_node, edge):
    distance = float('inf')
    nearest_point = None  
    for point in list(edge.coords):
        # Extract latitude and longitude from the point
        point_coords = (point[1] , point[0])  # (latitude, longitude)
        # Calculate distance
        current_distance = geodesic((pharmacy_node.y, pharmacy_node.x), point_coords).meters
        if distance > current_distance:
            distance = current_distance
            nearest_point = point
            
    return nearest_point


In [4]:
def process_pharmacie(G,osmnd,pharmacy_point):
    nearest_edge = ox.distance.nearest_edges(G, pharmacy_point.x, pharmacy_point.y)
    u , v = nearest_edge[:2]
    edge_data = G.get_edge_data(*nearest_edge)
    
    if 'geometry' in edge_data:
            #print('list -> : ',list(edge_data['geometry'].coords))
            l=[]
            edge_geom =LineString(list(edge_data['geometry'].coords).copy())
            #edge_geom =LineString(project_and_insert_point(edge_geom, pharmacy_point))
            a = find_nearest_point(pharmacy_point,edge_geom)
            b = list(edge_geom.coords)
            
            for i in b :
                l.append(i)
                if calculate_distance(a,i) < 0.1 : 
                    l.append((pharmacy_point.x,pharmacy_point.y))
                    break
            
            edge_geom = LineString(l)
            

    else:
        # Si l'edge n'a pas de géométrie, créez une ligne entre les points
        u , v = nearest_edge[:2]
        edge_geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y'])
                                ,projection_point_on_line(G.nodes[u]['x'], G.nodes[u]['y'],G.nodes[v]['x'], G.nodes[v]['y'], pharmacy_point.x, pharmacy_point.y)
                                , (pharmacy_point.x, pharmacy_point.y)])
    new_edge_data = {
     'reversed': True,
     'length': calculate_linestring_distance_in_meters(edge_geom.coords),
      'geometry' : edge_geom }
    add_pharmacy_node(G,osmnd,pharmacy_point)
    G.add_edge(u,osmnd, **new_edge_data)
    if v in G and u in G[v] and G[v][u] and edge_data['reversed']: 
        edge_data = G[v][u][0]
        if 'geometry' in edge_data:
            l=[]
            edge_geom =LineString(list(edge_data['geometry'].coords).copy())
            #edge_geom =LineString(project_and_insert_point(edge_geom, pharmacy_point))
            a = find_nearest_point(pharmacy_point,edge_geom)
            b = list(edge_geom.coords)
            
            for i in b :
                l.append(i)
                if calculate_distance(a,i) < 0.1 : 
                    l.append((pharmacy_point.x,pharmacy_point.y))
                    break
            
            edge_geom = LineString(l)
        else:

            # Si l'edge n'a pas de géométrie, créez une ligne entre les points
            u, v = u,v
            edge_geom = LineString([(G.nodes[u]['x'], G.nodes[u]['y'])
                                    ,projection_point_on_line(G.nodes[u]['x'], G.nodes[u]['y'],G.nodes[v]['x'], G.nodes[v]['y'], pharmacy_point.x, pharmacy_point.y)
                                    , (pharmacy_point.x, pharmacy_point.y)])
    new_edge_data = {
     'reversed': True,
     'length': calculate_linestring_distance_in_meters(edge_geom.coords),
      'geometry' : edge_geom }
    G.add_edge(v, osmnd, **new_edge_data)

In [5]:
G = ox.graph_from_place('Fès, Fès-Meknès, Maroc', network_type='drive')

In [6]:
2# Identifier les pharmacies en utilisant la nouvelle fonction
pharmacies = ox.geometries_from_place('Fès, Fès-Meknès, Maroc', tags={'amenity': 'pharmacy'})

C:\Users\pccc\AppData\Local\Temp\ipykernel_9100\2857674731.py:2: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  pharmacies = ox.geometries_from_place('Fès, Fès-Meknès, Maroc', tags={'amenity': 'pharmacy'})


In [7]:
pharmacy_points = [
    ( index[1],row['geometry'])
    for index, row in pharmacies.iterrows()
]


In [8]:
#for i in pharmacy_points:
#    print(i)

In [9]:
a = 1
for i,j in pharmacy_points:
    print(a,end='->')
    process_pharmacie(G,i,j)
    a+=1

1->osmid :  1866502248
2->osmid :  2328748922
3->osmid :  3718016810
4->osmid :  4606353707
5->osmid :  4681620692
6->osmid :  4698968089
7->osmid :  4711448391
8->osmid :  4711868990
9->osmid :  4711934790
10->osmid :  4711934894
11->osmid :  4712385991
12->osmid :  4712445890
13->osmid :  4712508653
14->osmid :  4724813489
15->osmid :  4725748199
16->osmid :  4729539594
17->osmid :  4730950544
18->osmid :  4738031421
19->osmid :  4738681626
20->osmid :  4740427422
21->osmid :  4741642523
22->osmid :  4744887621
23->osmid :  4746773124
24->osmid :  4746889021
25->osmid :  4753542728
26->osmid :  4757773462
27->osmid :  4759451623
28->osmid :  4759495424
29->osmid :  4763995654
30->osmid :  4772565121
31->osmid :  4774537683
32->osmid :  4788557121
33->osmid :  4788557323
34->osmid :  4797223421
35->osmid :  4801102022
36->osmid :  4801209322
37->osmid :  4802070122
38->osmid :  4803244621
39->osmid :  4803711989
40->osmid :  4805354921
41->osmid :  4805418523
42->osmid :  4805680322
4

In [10]:
import numpy as np
from shapely.geometry import LineString, Point

def project_and_insert_point(path_coords, point_coords):
    """
    Projette un point sur un chemin défini par une liste de coordonnées et insère le point projeté
    dans la liste des points du chemin à la bonne position.

    :param path_coords: Liste de tuples représentant les coordonnées du chemin [(x1, y1), (x2, y2), ...]
    :param point_coords: Tuple représentant les coordonnées du point (x, y)
    :return: Nouvelle liste de tuples représentant les coordonnées du chemin avec le point projeté inséré
    """
    path = LineString(path_coords)
    point = Point(point_coords)
    
    # Trouver la position projetée du point sur le chemin
    distance = path.project(point)
    projected_point = path.interpolate(distance)
    
    # Convertir le point projeté en tuple de coordonnées
    projected_coords = (projected_point.x, projected_point.y)
    
    # Trouver le segment du chemin où insérer le point projeté
    new_path_coords = []
    inserted = False
    for i in range(len(path_coords) - 1):
        segment_start = Point(path_coords[i])
        segment_end = Point(path_coords[i + 1])
        segment = LineString([segment_start, segment_end])
        
        new_path_coords.append(path_coords[i])
        
        # Vérifier si le point projeté doit être inséré dans ce segment
        if not inserted and segment.project(projected_point) < segment.length:
            new_path_coords.append(projected_coords)
            inserted = True
    
    new_path_coords.append(path_coords[-1])  # Ajouter le dernier point du chemin
    
    if not inserted:
        # Si le point projeté n'a pas été inséré dans les segments, on l'ajoute à la fin
        new_path_coords.append(projected_coords)
    print(projected_coords)
    return new_path_coords

# Exemple d'utilisation
chemin = [(0, 0), (1, 2), (2, 1), (3, 3)]  # Un chemin non linéaire
point = (2, 2)

nouveau_chemin = project_and_insert_point(chemin, point)
print("Le nouveau chemin avec le point projeté est :")
for coord in nouveau_chemin:
    print(coord)


(2.4, 1.7999999999999998)
Le nouveau chemin avec le point projeté est :
(0, 0)
(1, 2)
(2.4, 1.7999999999999998)
(2, 1)
(3, 3)


# Stocker le graphe dans Neo4j

In [11]:
from neo4j import GraphDatabase

# Fonction pour convertir le graphe OSMnx en données Neo4j
def convert_nodes_osmnx_graph_to_neo4j(G):
    # Connexion à la base de données Neo4j
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "bdsas2024"
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    # Créer une session Neo4j
    with driver.session(database="ProjetNeo4j") as session:
        # Parcourir chaque nœud du graphe OSMnx
        for node in G.nodes(data=True):
            # Préparer les propriétés du nœud
            properties = {
                'osmid': node[0],
                'y': node[1].get('y'),
                'x': node[1].get('x')
            }
            
            # Vérifier la présence de l'attribut 'pharmacy'
            if 'pharmacy' in node[1]:
                properties['pharmacy'] = node[1]['pharmacy']

            
            # Construire la requête Cypher pour créer le nœud
            query = "CREATE (a:Location {osmid: $osmid, y: $y, x: $x"
            if 'pharmacy' in properties:
                query += ", pharmacy: $pharmacy"
            query += "})"
            # Vérifiez les types de chaque propriété et exclure les non valides
            valid_properties = {k: v for k , v in properties.items() if not callable(v) and v is not None}
            # Affichez les propriétés valides pour le débogage
            
            # Exécuter la requête avec les propriétés du nœud
            session.run(query, **valid_properties)
    
    # Fermer la connexion
    driver.close()

# Appeler la fonction pour convertir le graphe
convert_nodes_osmnx_graph_to_neo4j(G)


In [12]:
# Fonction pour convertir le graphe OSMnx en données Neo4j
def convert_edges_osmnx_graph_to_neo4j(G):
    # Connexion à la base de données Neo4j
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "bdsas2024"
    driver = GraphDatabase.driver(uri, auth=(user, password))
    # Créer une session Neo4j
    with driver.session(database="ProjetNeo4j") as session:
        for u, v, k, data in G.edges(keys=True, data=True):
            # Préparer les propriétés de l'arête
            edge_properties = {
                'length': data.get('length', 0)
            }
            if 'geometry' in data:
                edge_properties['geometry'] = ""
                a = list(data['geometry'].coords)
                for i in a :
                    edge_properties['geometry'] = edge_properties['geometry']+'|'+ str(i[0]) +"#"+ str(i[1])
            else :
                edge_properties['geometry']="|"+str(G._node[u]['x']) +"#"+str(G._node[u]['y']) + "|" +str(G._node[v]['x']) +"#"+str(G._node[v]['y'])
            # Ajouter d'autres attributs de l'arête si disponibles
            if 'name' in data:
                edge_properties['name'] = data['name']
            if 'highway' in data:
                edge_properties['highway'] = data['highway']

            # Créer une relation dans Neo4j pour chaque bord OSMnx
            query = """
            MATCH (a:Location {osmid: $u}), (b:Location {osmid: $v})
            CREATE (a)-[:ROAD {length: $length"""
            if 'geometry' in edge_properties:
                query += ", geometry: $geometry"
            if 'name' in edge_properties:
                query += ", name: $name"
            if 'highway' in edge_properties:
                query += ", highway: $highway"
            query += "}]->(b)"

            session.run(query, u=u, v=v, **edge_properties)

    # Fermer la connexion
    driver.close()

# Appeler la fonction pour convertir le graphe
convert_edges_osmnx_graph_to_neo4j(G)
